In [40]:
import pandas as pd
import os
import statsmodels.api as sm
import numpy as np
from linearmodels.panel import PanelOLS

In [22]:
data = pd.read_excel(os.path.join('data', 'KF_data.xlsx'))

data['date'] = pd.to_datetime(data['date'], format='%Y%m')
data = data.set_index('date')
data = data.astype('float')

factors = data[['VWMe', 'SMB', 'HML']].values
rf = data[['RF']].values
stocks = data.iloc[:,6:].values

excess_returns = stocks - rf

In [23]:
excess_returns

array([[-0.86, -1.64, -0.86, ...,  1.78,  2.71,  0.34],
       [ 3.56,  0.96,  4.57, ...,  1.57,  5.39,  7.51],
       [-5.42,  2.85,  0.52, ..., -0.46, -0.53, -2.66],
       ...,
       [14.95, 14.  ,  9.79, ..., 12.82,  9.16, 12.72],
       [-3.15, -0.1 , -0.57, ...,  0.84,  0.72, -4.46],
       [ 1.55,  1.5 ,  1.75, ...,  2.74,  3.2 ,  0.61]])

In [24]:
excess_returns.shape

(1026, 23)

In [25]:
factors

array([[ 2.620e+00, -2.160e+00, -2.920e+00],
       [ 2.560e+00, -1.490e+00,  4.880e+00],
       [ 3.600e-01, -1.380e+00, -1.000e-02],
       ...,
       [ 1.153e+01,  3.530e+00, -9.000e-01],
       [-6.100e-01, -2.200e-01, -2.000e-02],
       [ 4.900e-01, -4.400e-01,  1.560e+00]])

In [26]:
factors.shape

(1026, 3)

## 1. Fama-Macbeth regression

### Time series OLS regression

$
y_{i,t} = \alpha_t + \beta_{1,t}x1_{i,t} + \beta_{2,t}x2_{i,t} + \beta_{3,t}x3_{i,t} + \epsilon_{i,t}
$

In [62]:
# Time series regressions
x = sm.add_constant(factors)
ts_res = sm.OLS(excess_returns, x).fit()
alpha = ts_res.params[0].T
beta = ts_res.params[1:].T

In [63]:
avgexcess_returns = np.mean(excess_returns, 0)
avgexcess_returns

array([ 1.00446394,  1.1528655 ,  1.37066277,  0.57311891,  0.93388889,
        1.02578947,  1.06244639,  1.18837232,  0.6638499 ,  0.86633528,
        0.96592593,  0.98119883,  1.12352827,  0.67624756,  0.73533138,
        0.82982456,  0.93158869,  1.02979532,  0.58362573,  0.59477583,
        0.64549708,  0.6832846 , -0.2641423 ])

### Cross-section OLS regression

Use the estimated time series of betas as independent variables:

$
y_{t} = \lambda_{1}\beta_{1, t} + \lambda_{2}\beta_{2,t} + \lambda_{3}\beta_{3,t} + \epsilon_{t}
$

In [64]:
avgexcess_returns = np.mean(excess_returns, 0)
beta.columns = ['VWMe', 'SMB', 'HML']
# Cross-section regression
cs_res = sm.OLS(avgexcess_returns.T, beta).fit()
risk_premia = cs_res.params

In [65]:
cs_res.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:                      y   R-squared (uncentered):                   0.918
Model:                            OLS   Adj. R-squared (uncentered):              0.905
Method:                 Least Squares   F-statistic:                              74.31
Date:                Tue, 27 Oct 2020   Prob (F-statistic):                    5.09e-11
Time:                        17:57:34   Log-Likelihood:                         -1.4916
No. Observations:                  23   AIC:                                      8.983
Df Residuals:                      20   BIC:                                      12.39
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
VWMe           0.5479      0.092      5.951      0.000       0.356       0.740
SMB            0.3872      0.122      3.175      0.005       0.133       0.642
HML            0.1907      0.147      1.298      0.209      -0.116       0.497
==============================================================================
Omnibus:                       37.924   Durbin-Watson:                   1.192
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              107.268
Skew:                          -2.948   Prob(JB):                     5.09e-24
Kurtosis:                      11.784   Cond. No.                         3.36
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2. Panel OLS regression with time effects

$
y_{i,t} = \alpha_t + \beta_{1}x1_{i,t} + \beta_{2}x2_{i,t} + \beta_{3}x3_{i,t} + \epsilon_{i,t}
$

In [89]:
panel_excess_returns = pd.DataFrame([excess_returns[i] - excess_returns.mean(axis=1)[i] for i in range(0, len(excess_returns.mean(axis=1)))])
panel_excess_returns.columns = data.iloc[:,6:].columns
panel_excess_returns.index = factors.index
panel_df = pd.merge(pd.concat([factors[[col]] - factors[[col]].mean() for col in factors.columns], axis=1)
                    , panel_excess_returns, right_index=True, left_index=True).reset_index()
panel_df.rename(columns={'index': 'date'}, inplace=True)
panel_df['date'] = [i for i in range(0, len(panel_df))]
panel_df

,date,VWMe,SMB,HML,S1V3,S1V4,S1V5,S2V1,S2V2,S2V3,...,S4V1,S4V2,S4V3,S4V4,S4V5,S5V1,S5V2,S5V3,S5V4,S5V5
0,0,2.00268,-2.401511,-3.302183,-2.184783,-2.964783,-2.184783,1.875217,1.715217,0.875217,...,1.745217,-0.304783,-0.254783,-0.994783,1.015217,1.635217,4.535217,0.455217,1.385217,-0.984783
1,1,1.94268,-1.731511,4.497817,0.734783,-1.865217,1.744783,-1.485217,-5.985217,-0.055217,...,-2.315217,1.034783,-1.145217,-0.945217,1.394783,-1.905217,1.024783,-1.255217,2.564783,4.684783
2,2,-0.25732,-1.621511,-0.392183,-4.764783,3.505217,1.175217,-2.384783,-1.894783,1.575217,...,2.295217,0.345217,-1.414783,1.985217,2.605217,-0.944783,4.085217,0.195217,0.125217,-2.004783
3,3,-4.04732,-0.201511,0.327817,-0.715652,3.124348,0.024348,-0.565652,-0.685652,-3.935652,...,3.214348,1.374348,0.754348,0.154348,-2.125652,-0.055652,-0.045652,0.874348,-1.505652,-2.725652
4,4,1.82268,-0.481511,-0.692183,-0.114783,-5.784783,-1.024783,-0.924783,-5.194783,1.605217,...,1.115217,-0.144783,1.425217,2.575217,-0.644783,1.885217,0.185217,-0.954783,1.125217,0.135217
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1021,1021,-6.47732,-3.481511,-1.872183,-1.681739,-1.661739,-1.761739,-0.831739,-1.351739,-1.381739,...,0.528261,0.748261,0.058261,-1.511739,1.558261,4.318261,4.528261,-0.411739,4.038261,-0.661739
1022,1022,-9.04732,-3.821511,-1.372183,-0.155217,-0.545217,-0.405217,-0.135217,-1.075217,-3.065217,...,1.664783,-1.505217,-0.915217,-0.755217,0.624783,3.804783,4.904783,1.984783,3.964783,-0.815217
1023,1023,10.91268,3.288489,-1.282183,1.240000,0.290000,-3.920000,3.480000,0.950000,3.580000,...,0.490000,2.190000,0.400000,-0.950000,-2.050000,-4.670000,-2.940000,-0.890000,-4.550000,-0.990000
1024,1024,-1.22732,-0.461511,-0.402183,-2.967826,0.082174,-0.387826,0.002174,-1.217826,0.382174,...,0.052174,-0.237826,1.422174,0.192174,-0.077826,-0.317826,1.012174,1.022174,0.902174,-4.277826


In [90]:
panel_df = panel_df.melt(['date', 'VWMe', 'SMB', 'HML'])
panel_df.rename(columns={'variable': 'stocks'}, inplace=True)
panel_df = panel_df.set_index(['stocks', 'date'])
panel_df.head()

VWMe       SMB       HML     value
stocks date                                       
S1V3   0     2.00268 -2.401511 -3.302183 -2.184783
       1     1.94268 -1.731511  4.497817  0.734783
       2    -0.25732 -1.621511 -0.392183 -4.764783
       3    -4.04732 -0.201511  0.327817 -0.715652
       4     1.82268 -0.481511 -0.692183 -0.114783

In [91]:
panel_res = PanelOLS(panel_df.value, panel_df[['VWMe', 'SMB', 'HML']], time_effects=True).fit()
print(panel_res)

                          PanelOLS Estimation Summary                           
Dep. Variable:                  value   R-squared:                        0.0000
Estimator:                   PanelOLS   R-squared (Between):              0.0000
No. Observations:               23598   R-squared (Within):              -0.0042
Date:                Tue, Oct 27 2020   R-squared (Overall):             -0.0042
Time:                        18:09:55   Log-likelihood                 -6.42e+04
Cov. Estimator:            Unadjusted                                           
                                        F-statistic:                      0.0000
Entities:                          23   P-value                           1.0000
Avg Obs:                       1026.0   Distribution:                 F(3,22569)
Min Obs:                       1026.0                                           
Max Obs:                       1026.0   F-statistic (robust):          3.696e-30
                            

C:\Users\daniel.oliveira\.conda\envs\quant2\lib\site-packages\linearmodels\panel\data.py:98: FutureWarning: is_categorical is deprecated and will be removed in a future version.  Use is_categorical_dtype instead
  if is_categorical(s):
